1. Setup & Target Files

In [1]:
"""
Notebook 10 – Data Dictionary (Processed)
Goal:
- Mensintesis kamus data untuk seluruh CSV di data/processed.
- QC ringkas per dataset (range EOM 2020–2025, missing, duplikasi index).
- Menulis keluaran ke CSV + Markdown (opsional).

Note on tooling:
- Beberapa boilerplate dibantu oleh ChatGPT (GPT-5 Thinking); semua hasil sudah diverifikasi dan disesuaikan manual.
"""

from pathlib import Path
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Any
from pandas.api.types import is_numeric_dtype, is_datetime64_any_dtype

PROC = Path("data/processed")
TBL  = Path("reports/tables");  TBL.mkdir(parents=True, exist_ok=True)

FILES = [
    PROC/"gold_monthly_clean_2020_2025.csv",
    PROC/"btc_monthly_close_2020_2025.csv",
    PROC/"merged_gold_btc_monthly_2020_2025.csv",
    PROC/"monthly_returns_gold_btc_2020_2025.csv",
]

# add all other processed CSVs (avoid duplicates & non-existent files)
seen = {p.resolve() for p in FILES}
for extra in PROC.glob("*.csv"):
    if extra.resolve() not in seen and extra.exists():
        FILES.append(extra)

FILES = [p for p in FILES if p.exists()]
print("Processed files:", [f.name for f in FILES])


Processed files: ['gold_monthly_clean_2020_2025.csv', 'btc_monthly_close_2020_2025.csv', 'merged_gold_btc_monthly_2020_2025.csv', 'monthly_returns_gold_btc_2020_2025.csv']


2. Function Profiler (Column & Dataset-Level)

In [2]:
def _infer_role(col: str) -> str:
    c = col.lower()
    if c == "date": return "key(time)"
    if "ret" in c:  return "feature(return)"
    if "usd" in c:  return "feature(level)"
    return "feature"

def _infer_unit(col: str) -> str:
    c = col.lower()
    if c == "date": return "YYYY-MM (EOM)"
    if "ret" in c:  return "log return (unitless)"
    if "usd" in c:  return "USD"
    return ""

def _infer_desc(file: str, col: str) -> str:
    f = file.lower()
    c = col.lower()
    if c == "date": return "End-of-month timestamp (EOM)"
    if f.startswith("gold") and c == "gold_usd":
        return "Monthly gold price (USD), end-of-month close"
    if f.startswith("btc") and c == "btc_usd":
        return "Monthly Bitcoin price (USD), end-of-month close"
    if "merged" in f and c in ("gold_usd","btc_usd"):
        return f"Merged {col} at EOM"
    if "returns" in f and "ret" in c:
        base = col.split("_")[0].title()
        return f"Month-over-month log return of {base}"
    return ""

def profile_file(path: Path) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Return (columns_profile, dataset_profile) for a CSV in processed/.
    - Normalizes 'Date' to EOM if present.
    - Computes column stats + dataset coverage w.r.t. 2020-01..2025-12.
    """
    df = pd.read_csv(path)

    # Time index normalization
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df = df.set_index("Date").sort_index()
        if isinstance(df.index, pd.DatetimeIndex):
            df.index = df.index.to_period("M").to_timestamp("M")

    # ---- Column-level ----
    rows: List[Dict[str, Any]] = []
    # column order: show Date first if available
    ordered_cols = (["Date"] if "Date" in df.reset_index().columns else []) + [c for c in df.columns if c != "Date"]

    for col in ordered_cols:
        if col == "Date" and isinstance(df.index, pd.DatetimeIndex):
            s = df.index.to_series(name="Date")
        else:
            s = (df[col] if col in df.columns else df.reset_index()[col])

        dtype    = str(s.dtype)
        non_null = int(s.notna().sum())
        nulls    = int(s.isna().sum())
        distinct = int(s.nunique(dropna=True))
        example  = s.dropna().iloc[0] if non_null else np.nan

        # safe min/max
        if is_numeric_dtype(s):
            vmin = float(np.nanmin(s.values)) if non_null else np.nan
            vmax = float(np.nanmax(s.values)) if non_null else np.nan
        elif is_datetime64_any_dtype(s):
            vmin = pd.to_datetime(s.min()) if non_null else pd.NaT
            vmax = pd.to_datetime(s.max()) if non_null else pd.NaT
        else:
            vmin = vmax = np.nan

        rows.append({
            "file": path.name,
            "column": col,
            "dtype": dtype,
            "non_null": non_null,
            "nulls": nulls,
            "distinct": distinct,
            "min": vmin,
            "max": vmax,
            "example": example,
            "unit": _infer_unit(col),
            "role": _infer_role(col),
            "description": _infer_desc(path.name, col),
            "source": "Processed (pipeline steps 02–07)",
            "transform": (
                "Hourly→EOM monthly close" if "btc" in path.name.lower() else
                "Monthly EOM; cleaned & filtered 2020–2025" if "gold" in path.name.lower() else
                "Inner-join on EOM index" if "merged" in path.name.lower() else
                "Log returns (Δ ln level)" if "returns" in path.name.lower() else
                ""
            )
        })

    columns_profile = pd.DataFrame(rows)

    # ---- Dataset-level ----
    if isinstance(df.index, pd.DatetimeIndex):
        start, end = df.index.min(), df.index.max()
        eom_expected = pd.period_range("2020-01", "2025-12", freq="M").to_timestamp("M")
        present = pd.Index(df.index.unique()).sort_values()
        missing = eom_expected.difference(present)
        coverage_rate = round(100.0 * (len(eom_expected) - len(missing)) / len(eom_expected), 2)
        dupe_idx = int(pd.Index(df.index).duplicated().sum())
    else:
        start = end = pd.NaT
        missing = pd.DatetimeIndex([])
        coverage_rate = np.nan
        dupe_idx = 0

    dataset_profile = pd.DataFrame([{
        "file": path.name,
        "rows": len(df),
        "n_cols": df.shape[1],
        "start": start if pd.notna(start) else "",
        "end": end if pd.notna(end) else "",
        "eom_missing": int(len(missing)),
        "coverage_%": coverage_rate,
        "dup_index": dupe_idx
    }])

    return columns_profile, dataset_profile


3. Build Data Dictionaries + Combine Records Manually

In [3]:
all_cols, all_ds = [], []
for p in FILES:
    cprof, dprof = profile_file(p)
    all_cols.append(cprof)
    all_ds.append(dprof)

data_dict = pd.concat(all_cols, ignore_index=True)
dataset_summary = pd.concat(all_ds, ignore_index=True)

# Optional override of notes
notes_path = TBL/"data_dictionary_notes.csv" 
if notes_path.exists():
    notes = pd.read_csv(notes_path)
    data_dict = data_dict.merge(notes, on=["file","column"], how="left", suffixes=("","_note"))
    for k in ("description","unit","role"):
        if f"{k}_note" in data_dict.columns:
            data_dict[k] = data_dict[f"{k}_note"].fillna(data_dict[k])
            data_dict.drop(columns=[f"{k}_note"], inplace=True)

# Tidy up
data_dict = (data_dict
             .sort_values(["file","role","column"], kind="stable")
             .reset_index(drop=True))
dataset_summary = dataset_summary.sort_values("file").reset_index(drop=True)

data_dict.head(8), dataset_summary

(                                     file    column           dtype  non_null  \
 0         btc_monthly_close_2020_2025.csv   BTC_USD         float64        70   
 1         btc_monthly_close_2020_2025.csv      Date  datetime64[ns]        70   
 2        gold_monthly_clean_2020_2025.csv  Gold_USD         float64        67   
 3        gold_monthly_clean_2020_2025.csv      Date  datetime64[ns]        67   
 4   merged_gold_btc_monthly_2020_2025.csv   BTC_USD         float64        67   
 5   merged_gold_btc_monthly_2020_2025.csv  Gold_USD         float64        67   
 6   merged_gold_btc_monthly_2020_2025.csv      Date  datetime64[ns]        67   
 7  monthly_returns_gold_btc_2020_2025.csv   BTC_USD         float64        66   
 
    nulls  distinct                  min                  max  \
 0      0        70              6423.61             116009.4   
 1      0        70  2020-01-31 00:00:00  2025-10-31 00:00:00   
 2      0        67              1560.67              3352.66   


4. Save CSV + Markdown

In [4]:
# --- Save CSV ---
dict_path = TBL/"data_dictionary.csv"
sum_path  = TBL/"dataset_summary.csv"
data_dict.to_csv(dict_path, index=False)
dataset_summary.to_csv(sum_path, index=False)
print("Saved →", dict_path.resolve())
print("Saved →", sum_path.resolve())

# --- Compact Markdown for Word/Report ---
try:
    import tabulate  # noqa: F401 # just to make sure it's available
    md_lines = ["# Data Dictionary (Processed)\n"]
    for f, g in data_dict.groupby("file"):
        md_lines.append(f"\n## {f}\n")
        sub = g[["column","dtype","unit","role","description"]].copy()
        md_lines.append(sub.to_markdown(index=False))
    md = "\n".join(md_lines)
    md_path = TBL/"data_dictionary.md"
    with open(md_path, "w", encoding="utf-8") as fh:
        fh.write(md)
    print("Saved →", md_path.resolve())
except Exception as e:
    print("Skip Markdown (install tabulate to enable): pip install tabulate")
    print("Reason:", e)

Saved → C:\Users\Noveno\OneDrive\CA1-BTC-Gold-Correlation\reports\tables\data_dictionary.csv
Saved → C:\Users\Noveno\OneDrive\CA1-BTC-Gold-Correlation\reports\tables\dataset_summary.csv
Saved → C:\Users\Noveno\OneDrive\CA1-BTC-Gold-Correlation\reports\tables\data_dictionary.md
